## Практика по DataFrame


Датасет представляет собой таблицу с информацией о 300 тысячах поездок за первые пять дней сентября 2018 года и включает в себя следующую информацию:

starttime — время начала поездки (дата, время);

stoptime — время окончания поездки (дата, время);

start station id — идентификатор стартовой стоянки;

start station name — название стартовой стоянки;

start station latitude, start station longitude — географическая широта и долгота стартовой стоянки;

end station id — идентификатор конечной стоянки;

end station name — название конечной стоянки;

end station latitude, end station longitude — географическая широта и долгота конечной стоянки;

bikeid — идентификатор велосипеда;

usertype — тип пользователя (Customer — клиент с подпиской на 24 часа или на три дня, Subscriber — подписчик с годовой арендой велосипеда);

birth year — год рождения клиента;

gender — пол клиента (0 — неизвестный, 1 — мужчина, 2 — женщина).

In [1]:
import pandas as pd
data = pd.read_csv('citibike-tripdata.csv', sep=',')

Задание 6.1

Найдите идентификатор самой популярной стартовой стоянки. Запишите идентификатор в виде целого числа.

In [2]:
print(data['start station id'].mode()[0])

281.0


***

Задание 6.2

Велосипед с каким идентификатором является самым популярным?

In [3]:
print(data['bikeid'].mode()[0])

33887


***

Задание 6.3

Какой тип клиентов (столбец usertype) является преобладающим — Subscriber или Customer? В качестве ответа запишите долю клиентов преобладающего типа среди общего количества клиентов. Ответ округлите до сотых.

In [4]:
mode_usertype = data['usertype'].mode()[0]
count_mode_user = data[data['usertype'] == mode_usertype].shape[0]
print(round(count_mode_user / data.shape[0], 2))

0.77


***

Задание 6.4

Кто больше занимается велоспортом — мужчины или женщины? В ответ запишите число поездок для той группы, у которой их больше.

In [5]:
male_count = data[data['gender'] == 1].shape[0]
female_count = data[data['gender'] == 0].shape[0]
print(max([male_count, female_count]))

183582


***

Задание 6.5

В первую очередь удалим лишнюю информацию из данных.
В наших данных присутствуют столбцы, которые дублируют информацию друг о друге: это столбцы с идентификатором и названием стартовой и конечной стоянки. Удалите признаки идентификаторов стоянок. Сколько столбцов осталось?

In [6]:
data.drop(['start station id', 'end station id'], axis=1, inplace=True)
print(data.shape[1])

12


***

Задание 6.6

Замените признак birth year на более понятный признак возраста клиента age. Годом отсчёта возраста выберите 2018 год. Столбец birth year удалите из таблицы. Сколько поездок совершено клиентами старше 60 лет?

In [7]:
data['age'] = 2018 - data['birth year']
data.drop(['birth year'], axis=1, inplace=True)
print(data[data['age'] > 60].shape[0])

11837


***

Задание 6.7

Создайте признак длительности поездки trip duration. Для этого вычислите интервал времени между временем окончания поездки (stoptime) и её началом (starttime). Сколько целых минут длилась поездка под индексом 3 в таблице?

In [8]:
data['starttime'] = pd.to_datetime(data['starttime'])
data['stoptime'] = pd.to_datetime(data['stoptime'])
data['trip duration'] = (data['stoptime'] - data['starttime'])
data.loc[3, 'trip duration']

Timedelta('0 days 00:07:16.837000')

***

Задание 6.8

Создайте «признак-мигалку» weekend, который равен 1, если поездка начиналась в выходной день (суббота или воскресенье), и 0 — в противном случае. Выясните, сколько поездок начиналось в выходные.

In [9]:
weekday = data['starttime'].dt.dayofweek
data['weekend'] = weekday.apply(lambda x: 1 if x ==5 or x == 6 else 0)
data['weekend'].sum()

115135

***

Задание 6.9

Создайте признак времени суток поездки time_of_day. Время суток будем определять из часа начала поездки. Условимся, что:

поездка совершается ночью (night), если её час приходится на интервал от 0 (включительно) до 6 (включительно) часов;

поездка совершается утром (morning), если её час приходится на интервал от 6 (не включительно) до 12 (включительно) часов;

поездка совершается днём (day), если её час приходится на интервал от 12 (не включительно) до 18 (включительно) часов;

поездка совершается вечером (evening), если её час приходится на интервал от 18 (не включительно) до 23 часов (включительно).

Во сколько раз количество поездок, совершённых днём, больше, чем количество поездок, совёршенных ночью, за представленный в данных период времени? Ответ округлите до целых.

In [10]:
def get_time_of_day(time):
    if 0 <= time <= 6:
        return 'night'
    elif 6 < time <= 12:
        return 'morning'
    elif 12 < time <= 18:
        return 'day'
    elif 18 < time <= 23:
        return 'evening'

data['time_of_day'] = data['starttime'].dt.hour.apply(get_time_of_day)
a = data[data['time_of_day'] == 'day'].shape[0]
b = data[data['time_of_day'] == 'night'].shape[0]
print(round(a / b))

9
